In [1]:
import os, re
import pandas as pd
import numpy as np
from loguru import logger

os.chdir("..")

In [2]:
# Configura il range di stagioni per la raccolta dati (es. 2026 per la stagione 2025/2026)
stag_iniziale = 2022
stag_finale = 2026

# Configurazioni aggiuntive
DATA_DIR = "data"
PROCESSED_DIR = os.path.join(DATA_DIR, "processed")

# Pesi stagioni
weights_per_year = {
    '2025_26': 1.0,
    '2024_25': 1.2,
    '2023_24': 0.6,
    '2022_23': 0.2
}

In [3]:
def load_dataset(data_folder: os.PathLike):
    # Carico i Dataset in un unico DF e rinomino le colonne in modo uniforme
    all_dfs = []
    final_df = pd.DataFrame()
    
    fixed_columns = ['Id', 'Nome']

    for file_name in os.listdir(data_folder):
            if file_name.startswith('Dataset_') and file_name.endswith('.bcsv'):
                match = re.match(r'Dataset_(\d{4})_(\d{2})\.bcsv', file_name)
                if match:
                    full_start_year = match.group(1)
                    short_end_year = match.group(2)
                    season_year_suffix = f"{full_start_year}_{short_end_year}"

                    file_path = os.path.join(data_folder, file_name)
                    logger.info(f"Sto leggendo il file: {file_path}")

                    try:
                        df = pd.read_csv(file_path)
                        df.columns = df.columns.str.strip()

                        new_columns = {}
                        for column in df.columns:
                            if column in fixed_columns:
                                new_columns[column] = column
                            else:
                                new_columns[column] = f"{column}_{season_year_suffix}"

                        renamed_df = df.rename(columns=new_columns)
                        all_dfs.append(renamed_df)
                        logger.info(f"File {file_name} letto e colonne rinominate.")

                    except Exception as e:
                        logger.error(f"Errore durante la lettura o elaborazione del file {file_name}: {e}. Salto questo file.")

    if not all_dfs:
        logger.error("Nessun file 'Dataset_XXXX_YY.bcsv' trovato o elaborato.")
        return final_df

    final_df = all_dfs[0]
    if len(all_dfs) > 1:
        for i in range(1, len(all_dfs)):
            final_df = pd.merge(final_df, all_dfs[i], on=['Id', 'Nome'], how='outer')
            
    logger.success("\nUnione di tutti i dataset completata.")
    return final_df

In [4]:
def missing_imputation(df: pd.DataFrame, range_start: int, range_end: int):
    # Imputazione dei valori NaN con 666 o "N.D."
    final_df = df.copy()
    
    historical_text_columns = ['R', 'RM', 'Squadra']
    numerical_prefixes = ['Qt.A', 'Qt.I', 'Diff.', 'Qt.A M', 'Qt.I M', 'Diff.M', 'FVM', 'FVM M',
                        'Rm', 'Pv', 'Mv', 'Fm', 'Gf', 'Gs', 'Rp', 'Rc', 'R+', 'R-', 'Ass', 'Amm', 'Esp', 'Au']

    years_range_for_suffix = range(range_start, range_end)

    for start_year in years_range_for_suffix:
        short_end_year = str(start_year + 1)[-2:]
        year_suffix = f"_{start_year}_{short_end_year}"

        for prefix in numerical_prefixes:
            numerical_column = f"{prefix}{year_suffix}"
            if numerical_column in final_df.columns:
                final_df[numerical_column] = pd.to_numeric(final_df[numerical_column], errors='coerce')
                final_df[numerical_column] = final_df[numerical_column].fillna(666)
                # Conversione a int solo se tutti i valori sono interi
                if pd.api.types.is_float_dtype(final_df[numerical_column]) and \
                   all(final_df[numerical_column].dropna().apply(lambda x: x == int(x))):
                    final_df[numerical_column] = final_df[numerical_column].astype(int)

        for col_text in historical_text_columns:
            text_column = f"{col_text}{year_suffix}"
            if text_column in final_df.columns:
                final_df[text_column] = final_df[text_column].fillna('N.D.')

    logger.success("Valori NaN nelle colonne numeriche riempiti con 666 e nelle colonne testuali con 'N.D.'.")
    return final_df
                

In [5]:
def calculate_perf_score(df: pd.DataFrame, suffix_cy: str):
    # Calcolo Indice_Beccalossi
    final_df = df.copy()

    MAX_SEASON_MATCHES = 38 # Numero massimo di partite in una stagione di Serie A

    final_df['Punteggio_Performance_Ponderato'] = 0.0

    for year_str, weight in weights_per_year.items():
        fm_col = f"Fm_{year_str}"
        pv_col = f"Pv_{year_str}"

        if fm_col in final_df.columns and pv_col in final_df.columns:
            # Sostituisci 666 con NaN per escluderli dal calcolo, poi riempi con 0 la performance
            temp_fm = final_df[fm_col].replace(666, np.nan)
            temp_pv = final_df[pv_col].replace(666, np.nan)

            # Calcola la performance solo se sia Fm che Pv sono disponibili
            performance_year = np.where(temp_fm.notna() & temp_pv.notna(), temp_fm * (temp_pv / MAX_SEASON_MATCHES), 0)
            final_df['Punteggio_Performance_Ponderato'] += performance_year * weight
        else:
            logger.warning(f"Attenzione: Colonne '{fm_col}' o '{pv_col}' non trovate per il calcolo del Fattore_Fantahack.")

    max_ppp = final_df['Punteggio_Performance_Ponderato'].max()
    if max_ppp > 0:
        final_df['Punteggio_Performance_Ponderato_Normalizzato'] = final_df['Punteggio_Performance_Ponderato'] / max_ppp
    else:
        final_df['Punteggio_Performance_Ponderato_Normalizzato'] = 0.0

    qt_a_col_cy = f"Qt.A_{suffix_cy}"
    if qt_a_col_cy in final_df.columns:
        # Sostituisci 666 con NaN per il calcolo, poi rimetti 666 alla fine se necessario
        current_quotations_temp = final_df[qt_a_col_cy].replace(666, np.nan)

        # Calcola Indice_Beccalossi solo per valori di quotazione validi (> 0)
        final_df['Indice_Beccalossi'] = np.where(
            current_quotations_temp.notna() & (current_quotations_temp > 0),
            final_df['Punteggio_Performance_Ponderato_Normalizzato'] / np.log(current_quotations_temp + 1),
            0.0 # Se la quotazione non è valida, Indice_Beccalossi è 0
        )
    else:
        logger.warning(f"Attenzione: Colonna '{qt_a_col_cy}' non trovata per il calcolo dell'Indice_Beccalossi.")
        final_df['Indice_Beccalossi'] = 0.0

    final_df = final_df.drop(columns=['Punteggio_Performance_Ponderato', 'Punteggio_Performance_Ponderato_Normalizzato'])
    logger.success("'Indice_Beccalossi' calcolato.")
    return final_df

In [6]:
def calculate_fenomeno(df: pd.DataFrame):
    final_df = df.copy()
    
    final_df['Media_Fantamedia_Ponderata'] = 0.0
    final_df['Media_Partite_Giocate_Ponderata'] = 0.0
    total_fantamedia_weights = 0.0
    total_matches_weights = 0.0

    for year_str, weight in weights_per_year.items():
        fm_col = f"Fm_{year_str}"
        pv_col = f"Pv_{year_str}"

        if fm_col in final_df.columns:
            temp_fm = final_df[fm_col].replace(666, np.nan)
            final_df['Media_Fantamedia_Ponderata'] = np.where(
                temp_fm.notna(),
                final_df['Media_Fantamedia_Ponderata'] + temp_fm * weight,
                final_df['Media_Fantamedia_Ponderata']
            )
            total_fantamedia_weights += weight

        if pv_col in final_df.columns:
            temp_pv = final_df[pv_col].replace(666, np.nan)
            final_df['Media_Partite_Giocate_Ponderata'] = np.where(
                temp_pv.notna(),
                final_df['Media_Partite_Giocate_Ponderata'] + temp_pv * weight,
                final_df['Media_Partite_Giocate_Ponderata']
            )
            total_matches_weights += weight

    if total_fantamedia_weights > 0: final_df['Media_Fantamedia_Ponderata'] /= total_fantamedia_weights
    else: final_df['Media_Fantamedia_Ponderata'] = 0.0 

    if total_matches_weights > 0: final_df['Media_Partite_Giocate_Ponderata'] /= total_matches_weights
    else: final_df['Media_Partite_Giocate_Ponderata'] = 0.0

    # Definizione soglie per il campo 'Fenomeno'
    valid_fm = final_df['Media_Fantamedia_Ponderata'][final_df['Media_Fantamedia_Ponderata'] > 0]
    valid_pv = final_df['Media_Partite_Giocate_Ponderata'][final_df['Media_Partite_Giocate_Ponderata'] > 0]

    high_fantamedia_threshold = valid_fm.quantile(0.75) if not valid_fm.empty else 6.5
    high_matches_threshold = valid_pv.quantile(0.75) if not valid_pv.empty else 20

    logger.info(f"Soglia Fantamedia Ponderata alta: > {high_fantamedia_threshold:.2f}")
    logger.info(f"Soglia Partite Giocate Ponderate alte: > {high_matches_threshold:.2f}")

    final_df['Fenomeno'] = 0
    top_player_condition = (final_df['Media_Fantamedia_Ponderata'] >= high_fantamedia_threshold) & \
                            (final_df['Media_Partite_Giocate_Ponderata'] >= high_matches_threshold) & \
                            (final_df['Media_Fantamedia_Ponderata'].notna()) & \
                            (final_df['Media_Partite_Giocate_Ponderata'].notna())

    final_df.loc[top_player_condition, 'Fenomeno'] = 1

    # Rimuovi la riga che elimina le colonne.
    # final_df = final_df.drop(columns=['Media_Fantamedia_Ponderata', 'Media_Partite_Giocate_Ponderata'])
    
    logger.success("Campo 'Fenomeno' calcolato.")
    return final_df

In [7]:
def calculate_goldenboy(df: pd.DataFrame, suffix_cy: str):
    # Calcola gli affari del listone (alte performance e prezzo basso)
    final_df = df.copy()
    
    goldenboy_col = 'Affarone'
    final_df[goldenboy_col] = 0
    
    qt_a_col_cy = f"Qt.A_{suffix_cy}"

    if qt_a_col_cy in final_df.columns:
        # Sostituisci 666 con NaN per il calcolo di unicorno
        current_quotations = final_df[qt_a_col_cy].replace(666, np.nan)
        low_price_threshold = current_quotations.quantile(0.50) if not current_quotations.dropna().empty else 10 
        
        valid_fantahack_factor = final_df['Indice_Beccalossi'][final_df['Indice_Beccalossi'] > 0]
        high_performance_threshold = valid_fantahack_factor.quantile(0.75) if not valid_fantahack_factor.empty else 0.1

        logger.info(f"Soglia prezzo basso (Qt.A_{suffix_cy}): < {low_price_threshold:.2f}")
        logger.info(f"Soglia performance alta (Indice_Beccalossi): > {high_performance_threshold:.2f}")

        price_condition = (current_quotations <= low_price_threshold) & (current_quotations.notna()) & (current_quotations > 0)
        performance_condition = (final_df['Indice_Beccalossi'] >= high_performance_threshold) & (final_df['Indice_Beccalossi'].notna())

        final_df.loc[price_condition & performance_condition, goldenboy_col] = 1

    logger.success("Campo 'Affarone' calcolato.")
    return final_df

In [8]:
def process_datasets(data_folder: os.PathLike, range_start: int, range_end: int):
    
    suffix_cy = f"{range_end - 1}_{str(range_end)[-2:]}"

    logger.info("Caricamento e unione dei Dataset preprocessati...")
    df = load_dataset(data_folder)

    logger.info("Gestione dei valori 'NaN'...")
    df = missing_imputation(df, range_start, range_end)

    logger.info("Calcolo Indice_Beccalossi...")
    df = calculate_perf_score(df, suffix_cy)

    logger.info("Calcolo campo 'Fenomeno'...")
    df = calculate_fenomeno(df)

    logger.info("Individuo gli 'Affaroni'...")
    df = calculate_goldenboy(df, suffix_cy)

    logger.success("Processing del dataset completato")
    return df

In [9]:
final_df = process_datasets(PROCESSED_DIR, 2022, 2026)

final_df.to_csv(os.path.join(PROCESSED_DIR, "Dataset_processed_full.bcsv"), index=False)
logger.success("Dataset processato ed esportato in 'Dataset_processed_full.bcsv'")

2025-08-29 23:08:45.500 | INFO     | __main__:process_datasets:5 - Caricamento e unione dei Dataset preprocessati...
2025-08-29 23:08:45.503 | INFO     | __main__:load_dataset:17 - Sto leggendo il file: data/processed/Dataset_2024_25.bcsv
2025-08-29 23:08:45.509 | INFO     | __main__:load_dataset:32 - File Dataset_2024_25.bcsv letto e colonne rinominate.
2025-08-29 23:08:45.510 | INFO     | __main__:load_dataset:17 - Sto leggendo il file: data/processed/Dataset_2022_23.bcsv
2025-08-29 23:08:45.514 | INFO     | __main__:load_dataset:32 - File Dataset_2022_23.bcsv letto e colonne rinominate.
2025-08-29 23:08:45.515 | INFO     | __main__:load_dataset:17 - Sto leggendo il file: data/processed/Dataset_2025_26.bcsv
2025-08-29 23:08:45.518 | INFO     | __main__:load_dataset:32 - File Dataset_2025_26.bcsv letto e colonne rinominate.
2025-08-29 23:08:45.519 | INFO     | __main__:load_dataset:17 - Sto leggendo il file: data/processed/Dataset_2023_24.bcsv
2025-08-29 23:08:45.521 | INFO     | __ma

In [10]:
final_df

,Id,R_2024_25,RM_2024_25,Nome,Squadra_2024_25,Qt.A_2024_25,Qt.I_2024_25,Diff._2024_25,Qt.A M_2024_25,Qt.I M_2024_25,...,R-_2023_24,Ass_2023_24,Amm_2023_24,Esp_2023_24,Au_2023_24,Indice_Beccalossi,Media_Fantamedia_Ponderata,Media_Partite_Giocate_Ponderata,Fenomeno,Affarone
0,3,N.D.,N.D.,Radunovic,N.D.,666,666,666,666,666,...,0,0,0,0,0,0.046476,0.800000,1.400000,0,0
1,4,P,Por,Sportiello,Milan,2,1,1,2,1,...,0,0,0,0,0,0.125550,3.229333,3.200000,0,0
2,11,N.D.,N.D.,Conti,N.D.,666,666,666,666,666,...,666,666,666,666,666,0.000000,0.400000,0.066667,0,0
3,21,N.D.,N.D.,D'alessandro,N.D.,666,666,666,666,666,...,666,666,666,666,666,0.000000,0.405333,0.400000,0,0
4,22,C,M;C,De Roon,Atalanta,16,8,8,16,9,...,0,5,10,0,0,0.327162,6.056667,23.000000,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,7246,N.D.,N.D.,Sala A.,N.D.,666,666,666,666,666,...,666,666,666,666,666,0.000000,0.000000,0.000000,0,0
1088,7249,N.D.,N.D.,Buksa,N.D.,666,666,666,666,666,...,666,666,666,666,666,0.000000,0.000000,0.000000,0,0
1089,7252,N.D.,N.D.,Stulic,N.D.,666,666,666,666,666,...,666,666,666,666,666,0.000000,0.000000,0.000000,0,0
1090,7253,N.D.,N.D.,Bella-Kotchap,N.D.,666,666,666,666,666,...,666,666,666,666,666,0.000000,0.000000,0.000000,0,0
